# Intent classification with regex I

You'll begin by implementing a very simple technique to recognize intents - looking for the presence of keywords.

A dictionary, keywords, has already been defined. It has the intents "greet", "goodbye", and "thankyou" as keys, and lists of keywords as the corresponding values. For example, keywords["greet"] is set to "["hello","hi","hey"].

Also defined is a second dictionary, responses, indicating how the bot should respond to each of these intents. It also has a default response with the key "default".

The function send_message(), along with the bot and user templates, have also already been defined. Your job in this exercise is to create a dictionary with the intents as keys and regex objects as values.

In [1]:
import re

keywords = {'greet': ['hello', 'hi', 'hey'], 'goodbye': ['bye', 'farewell'], 'thankyou': ['thank', 'thx']}

# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, keys in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile('|'.join(keys))
    
# Print the patterns
print(patterns)

{'greet': re.compile('hello|hi|hey'), 'goodbye': re.compile('bye|farewell'), 'thankyou': re.compile('thank|thx')}


# Intent classification with regex II

With your patterns dictionary created, it's now time to define a function to find the intent of a message.

In [2]:
user_template = 'USER : {0}'
bot_template = 'BOT : {0}'

responses = {'greet': 'Hello you! :)', 'goodbye': 'goodbye for now', 'thankyou': 'you are very welcome', 'default': 'default message'}
    
def send_message(message):
    print(user_template.format(message))
    response = respond(message)
    print(bot_template.format(response))

# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if re.search(pattern, message):
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]

# Send messages
send_message("hello!")
send_message("bye byeee")
send_message("thanks very much!")

USER : hello!
BOT : Hello you! :)
USER : bye byeee
BOT : goodbye for now
USER : thanks very much!
BOT : you are very welcome


# Entity extraction with regex

Now you'll use another simple method, this time for finding a person's name in a sentence, such as "hello, my name is David Copperfield".

You'll look for the keywords "name" or "call(ed)", and find capitalized words using regex and assume those are names. Your job in this exercise is to define a find_name() function to do this.

In [3]:
# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile('(name|call)')
    # Create a pattern for finding capitalized words
    name_pattern = re.compile('[A-Z]{1}[a-z]*')
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi there!"
    else:
        return "Hello, {0}!".format(name)

# Send messages
send_message("my name is David Copperfield")
send_message("call me Ishmael")
send_message("People call me Cassandra")

USER : my name is David Copperfield
BOT : Hello, David Copperfield!
USER : call me Ishmael
BOT : Hello, Ishmael!
USER : People call me Cassandra
BOT : Hello, People Cassandra!


# word vectors with spaCy

In this exercise you'll get your first experience with word vectors! You're going to use the ATIS dataset, which contains thousands of sentences from real people interacting with a flight booking system.

The user utterances are available in the list sentences, and the corresponding intents in labels.

Your job is to create a 2D array X with as many rows as there are sentences in the dataset, where each row is a vector describing that sentence.

In [10]:
#!pip install spacy
#!python -m spacy download en
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [11]:
import spacy
import numpy as np

sentences = [' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning', ' what flights are available from pittsburgh to baltimore on thursday morning', ' what is the arrival time in san francisco for the 755 am flight leaving washington', ' cheapest airfare from tacoma to orlando', ' round trip fares from pittsburgh to philadelphia under 1000 dollars', ' i need a flight tomorrow from columbus to minneapolis', ' what kind of aircraft is used on a flight from cleveland to dallas', ' show me the flights from pittsburgh to los angeles on thursday', ' all flights from boston to washington', ' what kind of ground transportation is available in denver', ' show me the flights from dallas to san francisco', ' show me the flights from san diego to newark by way of houston', ' what is the cheapest flight from boston to bwi', ' all flights to baltimore after 6 pm', ' show me the first class fares from boston to denver', ' show me the ground transportation in denver', ' all flights from denver to pittsburgh leaving after 6 pm and before 7 pm', ' i need information on flights for tuesday leaving baltimore for dallas dallas to boston and boston to baltimore', ' please give me the flights from boston to pittsburgh on thursday of next week', ' i would like to fly from denver to pittsburgh on united airlines', ' show me the flights from san diego to newark', ' please list all first class flights on united from denver to baltimore', ' what kinds of planes are used by american airlines', " i'd like to have some information on a ticket from denver to pittsburgh and atlanta", " i'd like to book a flight from atlanta to denver", ' which airline serves denver pittsburgh and atlanta', " show me all flights from boston to pittsburgh on wednesday of next week which leave boston after 2 o'clock pm", ' atlanta ground transportation', ' i also need service from dallas to boston arriving by noon', ' show me the cheapest round trip fare from baltimore to dallas']

# Load the spacy model: nlp
nlp = spacy.load('en_core_web_md')

# Calculate the length of sentences
n_sentences = len(sentences)

# Calculate the dimensionality of nlp
embedding_dim = nlp.vocab.vectors_length

# Initialize the array with zeros: X
X = np.zeros((n_sentences, embedding_dim))

# Iterate over the sentences
for idx, sentence in enumerate(sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X[idx, :] = doc.vector

# Intent classification with sklearn

An array X containing vectors describing each of the sentences in the ATIS dataset has been created for you, along with a 1D array y containing the labels. The labels are integers corresponding to the intents in the dataset. For example, label 0 corresponds to the intent atis_flight.

Now, you'll use the scikit-learn library to train a classifier on this same dataset. Specifically, you will fit and evaluate a support vector classifier.

In [ ]:
# TODO create X_train, y_train and X_test from ATIS dataset (https://www.kaggle.com/hassanamin/atis-intent-classification-using-svm-and-spacy)
# Import SVC
from sklearn.svm import SVC

# Create a support vector classifier
clf = SVC(C=1)

# Fit the classifier using the training data
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Count the number of correct predictions
n_correct = 0
for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
        n_correct += 1

print("Predicted {0} correctly out of {1} test examples".format(n_correct, len(y_test)))

# Using spaCy's entity recognizer

In this exercise, you'll use spaCy's built-in entity recognizer to extract names, dates, and organizations from search queries. The spaCy library has been imported for you, and its English model has been loaded as nlp.

Your job is to define a function called extract_entities(), which takes in a single argument message and returns a dictionary with the included entity types as keys, and the extracted entities as values. The included entity types are contained in a list called include_entities.